In [5]:
# Libraries import
import pandas as pd
from io import StringIO
import numpy as np
from tabulate import tabulate

during loading data, I encountered issue with too many commas in some lines, therefore I clean double commas and commas on the end of the line, if there is more than 13 values in the row.

In [33]:
# Data load
# Open the file and clean commas
cleaned_lines = []
with open("modelowanie_pricing_EH.csv", "r") as f:
    for line in f:
        if len(line.split(",")) == 13:
            cleaned_lines.append(line)
        else:
            cleaned_lines.append(line.replace(",,",",").rstrip(",\n"))

# Convert cleaned data into a StringIO object for pandas
cleaned_csv = StringIO("\n".join(cleaned_lines))

# Read into pandas
df_freq = pd.read_csv(cleaned_csv, index_col=0)

In [34]:
# Data overview
df_freq.head()

,IDpol,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region
0,1.0,1.0,0.10,D,5.0,0.0,55.0,50.0,B12,Regular,1217.0,R82
1,3.0,1.0,0.77,D,5.0,0.0,55.0,50.0,B12,Regular,1217.0,R82
2,5.0,1.0,0.75,B,6.0,2.0,52.0,50.0,B12,Diesel,54.0,R22
3,10.0,1.0,0.09,NaN,7.0,0.0,46.0,50.0,B12,Diesel,76.0,R72
4,11.0,1.0,0.84,B,7.0,0.0,46.0,50.0,B12,Diesel,76.0,R72


In [35]:
df_freq.describe()

,IDpol,ClaimNb,Exposure,VehPower,VehAge,DrivAge,BonusMalus,Density
count,6.780130e+05,678012.000000,678012.000000,678012.000000,678012.000000,678012.000000,678012.000000,678012.000000
mean,2.621857e+06,0.053245,0.528749,6.454632,7.044251,45.499102,59.761506,1792.420280
std,1.641783e+06,0.240115,0.364441,2.050907,5.666225,14.137445,15.636669,3958.649096
min,1.000000e+00,0.000000,0.002732,4.000000,0.000000,18.000000,50.000000,1.000000
25%,1.157951e+06,0.000000,0.180000,5.000000,2.000000,34.000000,50.000000,92.000000
50%,2.272152e+06,0.000000,0.490000,6.000000,6.000000,44.000000,50.000000,393.000000
75%,4.046274e+06,0.000000,0.990000,7.000000,11.000000,55.000000,64.000000,1658.000000
max,6.114330e+06,16.000000,2.010000,15.000000,100.000000,100.000000,230.000000,27000.000000


In [9]:
numerical = ["ClaimNb", "Exposure", "VehPower", "VehAge", "DrivAge", "BonusMalus", "Density"]
categorical = ["Area", "VehBrand", "VehGas", "Region"]

In [59]:
for col in categorical:
    aggregated = (
        df_freq.groupby(col)["IDpol"]
        .agg(count="count")
        .reset_index()
        .assign(share=lambda x: round(100 * x["count"] / len(df_freq), 2))
    )
    
    print("\n", tabulate(aggregated, headers="keys"))


     Area      count    share
--  ------  -------  -------
 0  A        103957    15.33
 1  B         75458    11.13
 2  C        191880    28.3
 3  D        151596    22.36
 4  E        137167    20.23
 5  F         17954     2.65

     VehBrand      count    share
--  ----------  -------  -------
 0  B1           162736    24
 1  B10           17707     2.61
 2  B11           13585     2
 3  B12          166023    24.49
 4  B13           12178     1.8
 5  B14            4047     0.6
 6  B2           159861    23.58
 7  B3            53395     7.88
 8  B4            25179     3.71
 9  B5            34753     5.13
10  B6            28548     4.21

     VehGas      count    share
--  --------  -------  -------
 0  Diesel     332135    48.99
 1  Regular    345877    51.01

     Region      count    share
--  --------  -------  -------
 0  R11         69790    10.29
 1  R21          3026     0.45
 2  R22          7994     1.18
 3  R23          8784     1.3
 4  R24        160600    23.69


Dataset contains 12 features of which:
 - one is a key of a row, doesn't have any predictive value and it will be removed
 - 7 are numerical
 - 4 are cathegorical

All numerical values seems to be very right skewed, what will impact outliers analysis. On the other hand both min and max values looks reasonable and logical. Cathegorical values are mostly unevenlly distributed among categories.

In [60]:
# removing IDpol
df_freq.drop('IDpol', axis=1, inplace=True)

In [61]:
# null values overview
df_freq.isnull().sum()

ClaimNb       1
Exposure      1
Area          1
VehPower      1
VehAge        1
DrivAge       1
BonusMalus    1
VehBrand      1
VehGas        1
Density       1
Region        1
dtype: int64

In [62]:
df_freq[df_freq.isnull().any(axis=1)]

,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region
3,1.0,0.09,NaN,7.0,0.0,46.0,50.0,B12,Diesel,76.0,R72
567,NaN,0.85,C,7.0,0.0,45.0,50.0,B12,Regular,309.0,R73
32341,0.0,0.32,C,7.0,11.0,47.0,72.0,B2,NaN,168.0,R91
45732,0.0,1.00,A,NaN,1.0,50.0,50.0,B3,Diesel,15.0,R24
71923,0.0,1.00,C,4.0,8.0,67.0,50.0,B5,Regular,432.0,NaN
84919,0.0,1.00,C,5.0,5.0,30.0,NaN,B1,Diesel,163.0,R24
99992,0.0,1.00,E,6.0,6.0,64.0,50.0,B2,Diesel,NaN,R82
173492,0.0,NaN,A,5.0,7.0,33.0,50.0,B13,Diesel,31.0,R24
184622,0.0,0.50,A,9.0,NaN,47.0,79.0,B5,Regular,7.0,R93
184711,1.0,0.50,D,7.0,10.0,NaN,50.0,B2,Regular,1440.0,R93


There are 11 rows with null values in dataset, which is approximatelly 0.0016% of the whole dataset. Pottential solution is to use some imputation method like KNNImputer or replacing those values with median. However, this amount of data won't have much impact on training model, so for the puropose of computing optimization I will simple delete rows with missing values

In [63]:
df_freq.dropna(inplace=True)
df_freq[df_freq.isnull().any(axis=1)]

,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region


In [64]:
# explorying outliers
def find_outliers(df: pd.DataFrame, threshold = 1.5, method="IQR"):
    numeric_df = df.select_dtypes(include=[np.number])

    results = {
        "Bounds (Min / Max)": [],
        "Total Outliers": [],
        "Upper Bound Outliers": [],
        "Lower Bound Outliers": []
    }

    if method == "IQR":
        q1s = numeric_df.quantile(0.25)
        q3s = numeric_df.quantile(0.75)
        IQRs = q3s - q1s

        min_bounds = q1s - threshold * IQRs
        max_bounds = q3s + threshold * IQRs
    elif method == "z_score":
        means = numeric_df.mean()
        stds = numeric_df.std()
        
        min_bounds = means - threshold * stds
        max_bounds = means + threshold * stds
    else:
        raise ValueError("valid method values are: [IQR, z_score]")

    column_names = []
    for col in numeric_df.columns:
        min_bond = min_bounds[col]
        max_bond = max_bounds[col]

        outliers = numeric_df[col]
        outliers_mask = (outliers < min_bond) | (outliers > max_bond)
        outliers_count = outliers_mask.sum()

        column_names.append(col)
        results["Bounds (Min / Max)"].append(f"{min_bond:.3f} / {max_bond:.3f}")
        results["Total Outliers"].append(outliers_count)
        results["Upper Bound Outliers"].append((outliers > max_bond).sum())
        results["Lower Bound Outliers"].append((outliers < min_bond).sum())

    if results["Total Outliers"]:
        df_results = pd.DataFrame(results, index=column_names).T
        print(tabulate(df_results, headers="keys"))
    else:
        print("No outliers found.")

In [65]:
# find_outliers_IQR(df_freq)
find_outliers(df_freq)
print("\n")
find_outliers(df_freq, threshold=2, method="z_score")

                      ClaimNb        Exposure        VehPower        VehAge            DrivAge         BonusMalus       Density
--------------------  -------------  --------------  --------------  ----------------  --------------  ---------------  --------------------
Bounds (Min / Max)    0.000 / 0.000  -1.035 / 2.205  2.000 / 10.000  -11.500 / 24.500  2.500 / 86.500  29.000 / 85.000  -2257.000 / 4007.000
Total Outliers        34057          0               35071           3114              1275            62384            77566
Upper Bound Outliers  34057          0               35071           3114              1275            62384            77566
Lower Bound Outliers  0              0               0               0                 0               0                0


                      ClaimNb         Exposure        VehPower        VehAge           DrivAge          BonusMalus       Density
--------------------  --------------  --------------  --------------  ---------------

Depends on a method, number of observations classified as outliers vary, but in general is quite significent. However, based on that values are within logical range it may be characteristic of features distribution in given dataset. Therefore it requires further exploration before coverying the issue.

In [ ]:
# TODO: rozkład zmiennych
# TODO: dodać claim frequency
# TODO: zbadać korelację
# TODO: zmienne ciągłe w zmiennych kategorycznych